## Output of moving StagePi

In [9]:
import numpy as np

# Define the voltage limits for channels X and Y
voltage_min_x = -20
voltage_max_x = 120
voltage_min_y = -20
voltage_max_y = 120

# Initialize position and voltages
position_x = np.linspace(0, 9, 10)  # 10 steps in X direction
position_y = np.linspace(0, 9, 10)  # 10 steps in Y direction
voltages_x = np.zeros((10, 10))  # Initialize the voltage grid for X
voltages_y = np.zeros((10, 10))  # Initialize the voltage grid for Y

# Simulate movement
for i, x in enumerate(position_x):
    for j, y in enumerate(position_y):
        # Calculate voltages for channels X and Y (for demonstration purposes)
        voltage_x = np.clip(x, voltage_min_x, voltage_max_x)
        voltage_y = np.clip(y, voltage_min_y, voltage_max_y)
        voltages_x[i, j] = voltage_x*5
        voltages_y[i, j] = voltage_y*3

        # Print current position and voltages
        print(f"Position (X={x}, Y={y}): Voltage X = {voltage_x} volts, Voltage Y = {voltage_y} volts")

# Convert voltages to lists for output
voltages_x_list = voltages_x.tolist()
voltages_y_list = voltages_y.tolist()

# Return the position grid and corresponding voltages for channels X and Y
return position_x.tolist(), position_y.tolist(), voltages_x_list, voltages_y_list


Position (X=0.0, Y=0.0): Voltage X = 0.0 volts, Voltage Y = 0.0 volts
Position (X=0.0, Y=1.0): Voltage X = 0.0 volts, Voltage Y = 1.0 volts
Position (X=0.0, Y=2.0): Voltage X = 0.0 volts, Voltage Y = 2.0 volts
Position (X=0.0, Y=3.0): Voltage X = 0.0 volts, Voltage Y = 3.0 volts
Position (X=0.0, Y=4.0): Voltage X = 0.0 volts, Voltage Y = 4.0 volts
Position (X=0.0, Y=5.0): Voltage X = 0.0 volts, Voltage Y = 5.0 volts
Position (X=0.0, Y=6.0): Voltage X = 0.0 volts, Voltage Y = 6.0 volts
Position (X=0.0, Y=7.0): Voltage X = 0.0 volts, Voltage Y = 7.0 volts
Position (X=0.0, Y=8.0): Voltage X = 0.0 volts, Voltage Y = 8.0 volts
Position (X=0.0, Y=9.0): Voltage X = 0.0 volts, Voltage Y = 9.0 volts
Position (X=1.0, Y=0.0): Voltage X = 1.0 volts, Voltage Y = 0.0 volts
Position (X=1.0, Y=1.0): Voltage X = 1.0 volts, Voltage Y = 1.0 volts
Position (X=1.0, Y=2.0): Voltage X = 1.0 volts, Voltage Y = 2.0 volts
Position (X=1.0, Y=3.0): Voltage X = 1.0 volts, Voltage Y = 3.0 volts
Position (X=1.0, Y=4

SyntaxError: 'return' outside function (3913575447.py, line 32)

In [30]:
import numpy as np
import random
import time

class FrameTrigger:
    def on(self):
        print("Frame trigger is now ON")

    def off(self):
        print("Frame trigger is now OFF")

# Assuming the farmerscanframe method is part of a class with set_pos and scanline methods defined
class Scanner:
    def __init__(self):
        # Initialize the frame trigger and other necessary components here
        self.frtrig = FrameTrigger()  # Initialize the frame trigger object

    def set_pos(self, pos, channel):
        # Set the position for the given channel
        print(f"Setting position to {pos} on channel {channel}")

    def scanline(self, channel, positions, dwell_time):
        # Scan a line for the given channel with the specified dwell time
        for pos in positions:
            self.set_pos(pos, channel)
            print(f"Scanning position {pos} on channel {channel}")
            time.sleep(dwell_time)

    def frtrig(self, state):
        # Simulate frame trigger on or off
        print(f"Frame trigger {'on' if state else 'off'}")

    def farmerscanframe(self, xpos, ypos, dwell_time=0.1):
        '''
        scan in x&z axis (y axis is perpendicular to the lens), using volt signal
        xpos -> horizontal line list containing start stop and step size, channel 1
        ypos -> vertical line list containing start stop and step size, channel 2
        '''
        if dwell_time < 0.1 :
            raise Exception("Minimum Dwell time should be more than 100ms or chnge blink trigger function")
        
        # setup scanning position arrays
        xnumber = abs(xpos[1]-xpos[0]) / xpos[2] + 1
        ynumber = abs(ypos[1]-ypos[0]) / ypos[2] + 1
        yposs = np.linspace(ypos[0],ypos[1],int(ynumber))
        xposs = np.linspace(xpos[0],xpos[1],int(xnumber))
        
        # move to the start point
        #self.set_pos(yposs[0], 2)
        #self.threshold(yposs[0],2)
        
        # start scanning
        self.frtrig.on()
        #time.sleep(0.01)
        self.frtrig.off()
        # scan like the farmer is plowing the field
        for j,y in enumerate(yposs):
            self.set_pos(y, 3)
            print("Y: {}".format(y))
            if j % 2 != 0: # flip direction for every seconid line
                self.scanline(1,np.flip(xposs), dwell_time)
            else:
                self.scanline(1,xposs, dwell_time)
                
            #yield self.get_pos(2)

        # frame scan ends
        self.frtrig.on()
        
        time.sleep(0.1)
        self.frtrig.off()
        # json cannot convert numpy arrays so make a list out of it instead
        return [xposs.tolist(), yposs.tolist()]

    # Your existing farmerscanframe method goes here

# Create an instance of the Scanner class
scanner = Scanner()

# Define the grid size
grid_size = 5

# Generate random starting positions within the grid
x_start = random.randint(0, grid_size-1)
y_start = random.randint(0, grid_size-1)

# Define the voltage step size
voltage_step = 1  # Adjust as needed for your setup

# Define the dwell time
dwell_time = 0.1  # Adjust as needed for your setup

# Define the scanning range for x and y positions
xpos = [x_start, x_start + (grid_size - 1) * voltage_step, voltage_step]
ypos = [y_start, y_start + (grid_size - 1) * voltage_step, voltage_step]

# Call the farmerscanframe method with the generated positions
scan_result = scanner.farmerscanframe(xpos, ypos, dwell_time)

# Output the scan result
print(f"Scan result: {scan_result}")


Frame trigger is now ON
Frame trigger is now OFF
Setting position to 1.0 on channel 3
Y: 1.0
Setting position to 1.0 on channel 1
Scanning position 1.0 on channel 1
Setting position to 2.0 on channel 1
Scanning position 2.0 on channel 1
Setting position to 3.0 on channel 1
Scanning position 3.0 on channel 1


Setting position to 4.0 on channel 1
Scanning position 4.0 on channel 1
Setting position to 5.0 on channel 1
Scanning position 5.0 on channel 1
Setting position to 2.0 on channel 3
Y: 2.0
Setting position to 5.0 on channel 1
Scanning position 5.0 on channel 1
Setting position to 4.0 on channel 1
Scanning position 4.0 on channel 1
Setting position to 3.0 on channel 1
Scanning position 3.0 on channel 1
Setting position to 2.0 on channel 1
Scanning position 2.0 on channel 1
Setting position to 1.0 on channel 1
Scanning position 1.0 on channel 1
Setting position to 3.0 on channel 3
Y: 3.0
Setting position to 1.0 on channel 1
Scanning position 1.0 on channel 1
Setting position to 2.0 on channel 1
Scanning position 2.0 on channel 1
Setting position to 3.0 on channel 1
Scanning position 3.0 on channel 1
Setting position to 4.0 on channel 1
Scanning position 4.0 on channel 1
Setting position to 5.0 on channel 1
Scanning position 5.0 on channel 1
Setting position to 4.0 on channel 3
Y: 4.0
Sett

In [17]:
# Code from gitlab E515 stagepi created by Prof. Tim and his research group modified it using tensorflow.

import tensorflow as tf
tf.compat.v1.enable_eager_execution()

import numpy as np
import random
import time

# Ensure that TensorFlow's eager execution is enabled


class FrameTrigger:
    def on(self):
        print("Frame trigger is now ON")

    def off(self):
        print("Frame trigger is now OFF")

class Scanner:
    def __init__(self):
        self.frtrig = FrameTrigger()  # Initialize the frame trigger object

    def set_pos(self, pos, channel):
        # Calculate the voltage based on the position
        voltage = tf.multiply(5.0, pos)  # This is a placeholder, replace with your actual voltage calculation
        tf.print(f"Setting position to {pos} (Voltage: ", voltage, "V) on channel ", channel)

    def scanline(self, channel, positions, dwell_time):
        for pos in positions:
            self.set_pos(pos, channel)
            time.sleep(dwell_time)

    def farmerscanframe(self, xpos, ypos, dwell_time=0.1):
        if dwell_time < 0.1:
            raise Exception("Minimum Dwell time should be more than 100ms or change blink trigger function")
        
        xnumber = abs(xpos[1] - xpos[0]) / xpos[2] + 1
        ynumber = abs(ypos[1] - ypos[0]) / ypos[2] + 1
        yposs = tf.linspace(ypos[0], ypos[1], int(ynumber))
        xposs = tf.linspace(xpos[0], xpos[1], int(xnumber))
        
        self.frtrig.on()
        self.frtrig.off()
        for j, y in enumerate(yposs):
            self.set_pos(y, 3)
            if j % 2 != 0:
                self.scanline(1, tf.reverse(xposs, axis=[0]), dwell_time)
            else:
                self.scanline(1, xposs, dwell_time)

        self.frtrig.on()
        time.sleep(0.1)
        self.frtrig.off()
        return [xposs.numpy().tolist(), yposs.numpy().tolist()]

scanner = Scanner()

grid_size = 5
x_start = 0
y_start = random.randint(0, 5)
voltage_step = 2
dwell_time = 0.1

xpos = [x_start, x_start + (grid_size - 1) * voltage_step, voltage_step]
ypos = [y_start, y_start + (grid_size - 1) * voltage_step, voltage_step]

[x,y] = scanner.farmerscanframe(xpos, ypos, dwell_time)
print(f"Scan result: {scan_result}")

ValueError: tf.enable_eager_execution must be called at program startup.

In [19]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

import random
import time

class FrameTrigger:
    def on(self):
        print("Frame trigger is now ON")

    def off(self):
        print("Frame trigger is now OFF")

class Scanner:
    def __init__(self):
        self.frtrig = FrameTrigger()  # Initialize the frame trigger object

    def set_pos(self, pos, channel):
        # Calculate the voltage based on the position
        voltage = tf.multiply(5.0, pos)  # This is a placeholder, replace with your actual voltage calculation
        tf.print(f"Setting position to {pos} (Voltage: ", voltage, "V) on channel ", channel)

    def scanline(self, channel, positions, dwell_time):
        for pos in positions:
            self.set_pos(pos, channel)
            time.sleep(dwell_time)

    def farmerscanframe(self, xpos, ypos, dwell_time=0.1):
        if dwell_time < 0.1:
            raise Exception("Minimum Dwell time should be more than 100ms or change blink trigger function")
        
        xnumber = abs(xpos[1] - xpos[0]) / xpos[2] + 1
        ynumber = abs(ypos[1] - ypos[0]) / ypos[2] + 1
        yposs = tf.linspace(ypos[0], ypos[1], int(ynumber))
        xposs = tf.linspace(xpos[0], xpos[1], int(xnumber))
        
        self.frtrig.on()
        self.frtrig.off()
        for j, y in enumerate(yposs):
            self.set_pos(y, 3)
            if j % 2 != 0:
                self.scanline(1, tf.reverse(xposs, axis=[0]), dwell_time)
            else:
                self.scanline(1, xposs, dwell_time)

        self.frtrig.on()
        time.sleep(0.1)
        self.frtrig.off()
        return [xposs.numpy().tolist(), yposs.numpy().tolist()]

scanner = Scanner()

grid_size = 5
x_start = 0
y_start = random.randint(0, 5)
voltage_step = 2
dwell_time = 0.1

xpos = [x_start, x_start + (grid_size - 1) * voltage_step, voltage_step]
ypos = [y_start, y_start + (grid_size - 1) * voltage_step, voltage_step]

[x,y] = scanner.farmerscanframe(xpos, ypos, dwell_time)
print(f"Scan result: {[x,y]}")


ValueError: tf.enable_eager_execution must be called at program startup.